In [ ]:
import os

# Changes the current path to find the source files

current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()

In [ ]:
from utils import create_all_folders, Folders

create_all_folders()

### Get the cropping limits

In [ ]:
from utils import create_folder, get_file_base_name
from geojson_conversions import open_geojson_feature_collection

from data_processing import (
    find_annots_repartition,
    crop_annots_into_limits,
    annots_coordinates_to_local,
    save_annots_per_image,
    crop_all_images_from_annotations_folder,
    ImageData,
    get_cropping_limits,
)
from lidar_preprocessing import (
    download_lidar_names_shapefile,
    get_lidar_files_from_image,
    download_and_remove_overlap_geotiles,
    create_full_lidar,
)
from rgb_preprocessing import download_rgb_image_from_polygon
from chm import compute_chm, compute_slices_chm
import geojson
import numpy as np

In [ ]:
# Define tile size and OVERLAP
# TILE_SIZE = 1920  # Size of each tile
# OVERLAP = 480  # Overlap between tiles
TILE_SIZE = 640  # Size of each tile
OVERLAP = 0  # Overlap between tiles

In [ ]:
# Annotations file to use:

annotations_file_name = "122000_484000.geojson"

In [ ]:
annotations_path = os.path.join(Folders.FULL_ANNOTS.value, annotations_file_name)

annotations = open_geojson_feature_collection(annotations_path)

full_image_path_tif = download_rgb_image_from_polygon(annotations["bbox"])[0]

In [ ]:
from typing import Sequence


image_data = ImageData(full_image_path_tif)

shapefile_path = download_lidar_names_shapefile()

GEOTILES_OVERLAP = 20
intersection_file_names = get_lidar_files_from_image(image_data, shapefile_path, GEOTILES_OVERLAP)
intersection_file_paths = download_and_remove_overlap_geotiles(
    intersection_file_names, GEOTILES_OVERLAP
)

full_lidar_path, full_lidar_filtered_path = create_full_lidar(intersection_file_paths, image_data)

RESOLUTION = 0.08

resolution = RESOLUTION
# full_chm_path = os.path.join(
#     Folders.CHM.value,
#     f"{round(resolution*100)}cm",
#     "unfiltered",
#     "full",
#     f"{image_data.coord_name}.tif",
# )
# create_folder(os.path.dirname(full_chm_path))
# compute_chm(
#     full_lidar_path,
#     full_chm_path,
#     image_data.width_pixel,
#     image_data.height_pixel,
#     resolution,
#     verbose=True,
# )

z_tops: Sequence[float] = [1, 2, 3, 5, 7, 10, 15, 20, np.inf]
z_limits_list = [(-np.inf, z_top) for z_top in z_tops]
full_chm_filtered_paths = [
    os.path.join(
        Folders.CHM.value,
        f"{round(resolution*100)}cm",
        "filtered",
        f"{str(bottom)}_{str(top)}",
        "full",
        f"{image_data.coord_name}.tif",
    )
    for (bottom, top) in z_limits_list
]
for folder_path in full_chm_filtered_paths:
    create_folder(os.path.dirname(folder_path))
compute_slices_chm(
    laz_file_name=full_lidar_path,
    output_tif_paths=full_chm_filtered_paths,
    resolution=resolution,
    z_limits_list=z_limits_list,
    skip_if_file_exists=True,
)

full_images_folders_paths = [
    os.path.dirname(folder_path) for folder_path in full_chm_filtered_paths
]
full_images_folders_paths.append(os.path.dirname(image_data.path))

In [ ]:
from typing import List
from tqdm.notebook import tqdm
from tifffile import tifffile


cropping_limits_x, cropping_limits_y = get_cropping_limits(full_image_path_tif, TILE_SIZE, OVERLAP)
visibility_threshold = 0.2
annots_repartition = find_annots_repartition(
    cropping_limits_x, cropping_limits_y, annotations, image_data, visibility_threshold
)
crop_annots_into_limits(annots_repartition)
annots_coordinates_to_local(annots_repartition)

output_image_prefix = get_file_base_name(full_image_path_tif)
annotations_output_directory = os.path.join(Folders.CROPPED_ANNOTS.value, output_image_prefix)
save_annots_per_image(
    annots_repartition,
    annotations_output_directory,
    full_image_path_tif,
    clear_if_not_empty=True,
)

images_output_directory = os.path.join(Folders.CROPPED_IMAGES.value, output_image_prefix)
cropped_images_folders_paths = crop_all_images_from_annotations_folder(
    annotations_output_directory,
    full_images_folders_paths=full_images_folders_paths,
    clear_if_not_empty=False,
    remove_unused=True,
)

print(cropped_images_folders_paths)


def merge_all_chms(cropped_images_folders_paths: List[str]):
    if len(cropped_images_folders_paths) == 0:
        raise ValueError("cropped_images_folders_paths is empty.")

    folder_path_list = cropped_images_folders_paths[0].split(os.path.sep)
    output_folder_path = os.path.join(
        os.path.sep.join(folder_path_list[:-3]),
        "merged",
        os.path.sep.join(folder_path_list[-2:]),
    )

    create_folder(output_folder_path)

    cropped_chms_folders_paths = [
        folder_path
        for folder_path in cropped_images_folders_paths
        if Folders.CHM.value in folder_path
    ]

    for image_name in tqdm(
        os.listdir(cropped_images_folders_paths[0]), desc="Merging CHMs", leave=False
    ):
        all_images_paths = [
            os.path.join(folder_path, image_name) for folder_path in cropped_chms_folders_paths
        ]
        output_path = os.path.join(output_folder_path, image_name)

        # Load images as NumPy arrays
        images = [tifffile.imread(image_path) for image_path in all_images_paths]

        # Stack images along a new axis to create a multi-channel image
        multi_channel_image = np.stack(images, axis=0)

        print(f"{multi_channel_image.shape = }")

        # Save the result
        tifffile.imwrite(output_path, multi_channel_image)


merge_all_chms(cropped_images_folders_paths)